In [1]:
from __future__ import division
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from nets import inception_utils, inception_v3

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#load data
base_skin_dir = os.path.join('./Data/')
slim = tf.contrib.slim

tile_df = pd.read_csv(os.path.join(base_skin_dir, 'hmnist_28_28_L.csv'))
input_images = np.reshape(tile_df.values[:,:-1],[-1,28,28,1]).astype(np.float32)
input_labels = tile_df.values[:,-1]
input_size = np.shape(input_images)[0]

images = tf.constant(input_images[:9984], dtype=tf.float32)
labels = tf.constant(input_labels[:9984])
labels = tf.one_hot(labels, depth=7)

In [4]:
def get_checkpoint_init_fn():
    # Load from .ckpt file
    variables_to_restore = slim.get_variables_to_restore()
    global_step_reset = tf.assign(tf.train.get_or_create_global_step(), 0)
    slim_init_fn = slim.assign_from_checkpoint_fn("./inception_v3.ckpt",variables_to_restore,ignore_missing_vars=True)
    return slim_init_fn

In [5]:
inception_v3.inception_v3_arg_scope()

{'<function convolution2d at 0x127027aa0>': {'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
  'normalizer_fn': <function tensorflow.contrib.layers.python.layers.layers.batch_norm>,
  'normalizer_params': {'decay': 0.9997,
   'epsilon': 0.001,
   'fused': None,
   'scale': False,
   'updates_collections': 'update_ops'},
  'weights_initializer': <function tensorflow.contrib.layers.python.layers.initializers._initializer>,
  'weights_regularizer': <function tensorflow.contrib.layers.python.layers.regularizers.l2>},
 '<function fully_connected at 0x127029848>': {'weights_regularizer': <function tensorflow.contrib.layers.python.layers.regularizers.l2>}}

In [6]:
g = tf.Graph()
with g.as_default():
    # Split up data into batches
    dataset = tf.data.Dataset.from_tensor_slices((input_images,input_labels)).batch(64)
    iterator = dataset.make_one_shot_iterator()
    image, label = iterator.get_next()
    label = tf.one_hot(label, depth=7)
    # Creat inception_v3
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        logits, end_points = inception_v3.inception_v3(image, 7, "Mixed_5d", create_aux_logits=False)
    loss = tf.losses.softmax_cross_entropy(label, logits)
    learning_rate = tf.train.exponential_decay(1e-04, tf.train.get_or_create_global_step(), input_size / 64 * 2.5 , 0.94)
    opt = tf.train.GradientDescentOptimizer(learning_rate)
    train_tensor = slim.learning.create_train_op(loss, optimizer=opt)
    # Creat Summary
    slim.summaries.add_scalar_summary(loss, 'cross_entropy_loss', 'losses')
    slim.summaries.add_scalar_summary(learning_rate, 'learning_rate', 'training')
    
    # Train
    slim.learning.train(
        train_tensor,
        "./saved_model/",
        log_every_n_steps=300,
        graph=g,
        save_summaries_secs=300,
        save_interval_secs=600,
        init_fn=get_checkpoint_init_fn(),
        global_step=tf.train.get_global_step())

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from ./inception_v3.ckpt
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [3,3,1,32] rhs shape= [3,3,3,32]
	 [[node save/Assign_3 (defined at /Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/variables.py:748)  = Assign[T=DT_FLOAT, _class=["loc:@InceptionV3/Conv2d_1a_3x3/weights"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](InceptionV3/Conv2d_1a_3x3/weights, save/RestoreV2:3)]]

Caused by op u'save/Assign_3', de

InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [3,3,1,32] rhs shape= [3,3,3,32]
	 [[node save/Assign_3 (defined at /Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/variables.py:748)  = Assign[T=DT_FLOAT, _class=["loc:@InceptionV3/Conv2d_1a_3x3/weights"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](InceptionV3/Conv2d_1a_3x3/weights, save/RestoreV2:3)]]

Caused by op u'save/Assign_3', defined at:
  File "/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/runpy.py", line 72, in _ru